## Basic work with Spark

### import neccessary packages

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

### Spark initialization

In [2]:
spark = SparkSession.builder \
                    .appName("LAB5") \
                    .getOrCreate()

# Перевіримо чи ініціалізувалася спарк-сесія
print(f"Spark Version: {spark.version}")

Spark Version: 3.5.0


### Create simple DataFrame

In [7]:
data = [
    ("Ukraine", "Kyiv", 2_952_000),
    ("France", "Paris", 2_103_000),
    ("China", "Beijing", 21_705_000),
    ("USA", "Washington, D.C.", 331_000_000),
    ("Japan", "Tokyo", 125_800_000),
    ("UK", "London", 67_886_000)
]
columns = ["Country", "Capital", "Population"]

df = spark.createDataFrame(data, schema=columns)

# Перевіримо створений DataFrame
df.show()
print(df.dtypes)

+-------+----------------+----------+
|Country|         Capital|Population|
+-------+----------------+----------+
|Ukraine|            Kyiv|   2952000|
| France|           Paris|   2103000|
|  China|         Beijing|  21705000|
|    USA|Washington, D.C.| 331000000|
|  Japan|           Tokyo| 125800000|
|     UK|          London|  67886000|
+-------+----------------+----------+

[('Country', 'string'), ('Capital', 'string'), ('Population', 'bigint')]


### Make simple operation

In [4]:
# Відсортуємо країни за населення (за спаданням)
df_sorted = df.orderBy(col("Population").desc())
df_sorted.show()

+-------+----------------+----------+
|Country|         Capital|Population|
+-------+----------------+----------+
|    USA|Washington, D.C.| 331000000|
|  Japan|           Tokyo| 125800000|
|     UK|          London|  67886000|
|  China|         Beijing|  21705000|
|Ukraine|            Kyiv|   2952000|
| France|           Paris|   2103000|
+-------+----------------+----------+



In [5]:
# Знайдемо країну з найбільшою столицею за населенням
name_of_country_with_greatest_capital_by_population = df_sorted.first()['Country']
print(name_of_country_with_greatest_capital_by_population)

USA


In [6]:
# Додамо стовпець з категорією населення: < 25 млн - Small; 25-150 млн - Medium; > 150 млн - Large
df_with_category = df_sorted.withColumn(
    "Population_Category",
    when(col("Population") < 25_000_000, "Small")
    .when((col("Population") >= 25_000_000) & (col("Population") <= 150_000_000), "Medium")
    .otherwise("Large")
)

df_with_category.show()

+-------+----------------+----------+-------------------+
|Country|         Capital|Population|Population_Category|
+-------+----------------+----------+-------------------+
|    USA|Washington, D.C.| 331000000|              Large|
|  Japan|           Tokyo| 125800000|             Medium|
|     UK|          London|  67886000|             Medium|
|  China|         Beijing|  21705000|              Small|
|Ukraine|            Kyiv|   2952000|              Small|
| France|           Paris|   2103000|              Small|
+-------+----------------+----------+-------------------+



In [8]:
# Зупиняємо роботу спарк сесії
spark.stop()